In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

Loading Model

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2-1.5B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Qwen2 patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


Loading Dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json('/content/structured_ans.json')

In [ ]:
df.head()

,data
1,Below is an input that describes a question or...
10,Below is an input that describes a question or...
1001,Below is an input that describes a question or...
1002,Below is an input that describes a question or...
1003,Below is an input that describes a question or...


In [ ]:
df['data'][1]

"Below is an input that describes a question or task. Write a response that appropriately completes the request.\n\n###  Input:What is the designation of the US Air Force's fifth-generation stealth fighter jet? \n\n### Response:The Lockheed Martin/Boeing F-22 Raptor<|im_end|>"

In [ ]:
from datasets import Dataset

In [ ]:
df = df.reset_index()

In [ ]:
df.head()

,index,data
0,1,Below is an input that describes a question or...
1,10,Below is an input that describes a question or...
2,1001,Below is an input that describes a question or...
3,1002,Below is an input that describes a question or...
4,1003,Below is an input that describes a question or...


In [ ]:
df2 = df.drop(columns=['index'])

In [ ]:
df2

,data
0,Below is an input that describes a question or...
1,Below is an input that describes a question or...
2,Below is an input that describes a question or...
3,Below is an input that describes a question or...
4,Below is an input that describes a question or...
...,...
4729,Below is an input that describes a question or...
4730,Below is an input that describes a question or...
4731,Below is an input that describes a question or...
4732,Below is an input that describes a question or...


In [ ]:
final_dataset = Dataset.from_pandas(df2)

In [ ]:
final_dataset['data'][0:5]

["Below is an input that describes a question or task. Write a response that appropriately completes the request.\n\n###  Input:What is the designation of the US Air Force's fifth-generation stealth fighter jet? \n\n### Response:The Lockheed Martin/Boeing F-22 Raptor<|im_end|>",
 'Below is an input that describes a question or task. Write a response that appropriately completes the request.\n\n###  Input:What factors led to the decrease in F-22 production rates? \n\n### Response:The high production costs, the reduced demand for air-to-air combat capabilities, and the simultaneous development of the F-35 fighter jet were the key reasons for the reduction in F-22 production<|im_end|>',
 'Below is an input that describes a question or task. Write a response that appropriately completes the request.\n\n###  Input:What are the key factors that determine a Qualified (Q) performance in the Flight Leadership (if applicable) area of 3.3.25? \n\n### Response:A Qualified (Q) performance in Flight

In [ ]:
shuffled_dataset = final_dataset.shuffle(seed=42)

In [ ]:
shuffled_dataset['data'][0:5]

['Below is an input that describes a question or task. Write a response that appropriately completes the request.\n\n###  Input:What type of radar is used in aircraft? \n\n### Response:The AN/APG-77 or AN/APG-77(V)1 is a type of AESA radar used in aircraft<|im_end|>',
 'Below is an input that describes a question or task. Write a response that appropriately completes the request.\n\n###  Input:What is the difference between satisfactory and limited knowledge ratings in Area 34: Instrument Interpretation? \n\n### Response:The Q rating represents satisfactory knowledge, whereas the Q- rating represents limited knowledge with minor errors or omissions<|im_end|>',
 'Below is an input that describes a question or task. Write a response that appropriately completes the request.\n\n###  Input:What is an example of a U rating in Area 67, specifically in the context of a missed approach or climb out? \n\n### Response:A U rating indicates that the aircrew must significantly deviate from or fail 

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 18464768 | total: 1795552768 | Percentage: 1.0284%


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = shuffled_dataset,
    dataset_text_field = "data",
    max_seq_length = max_seq_length,

    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,

        num_train_epochs=1,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map:   0%|          | 0/4734 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,734 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 148
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,1.210600
2,1.235600
3,1.245100
4,1.236100
5,1.213700
6,1.227300
7,1.097900
8,1.208300
9,1.147300
10,1.171800


In [ ]:
prompt = """Below is an input that describes a question or task. Write a response that appropriately completes the request.

### Input:
{}

### Response:
{}"""

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "What is the desired altitude tolerance for the more specific altitude criterion (Q-)", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['Below is an input that describes a question or task. Write a response that appropriately completes the request.\n\n### Input:\nWhat is the desired altitude tolerance for the more specific altitude criterion (Q-)\n\n### Response:\n±200 feet<|im_end|>']

In [ ]:
model.push_to_hub("APP04/qwen-1.5B", token = "")
tokenizer.push_to_hub("APP04/qwen-1.5B", token = "")

README.md:   0%|          | 0.00/590 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/73.9M [00:00<?, ?B/s]

Saved model to https://huggingface.co/APP04/qwen-1.5B


In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "What does Qualified-minus (Q-) performance in Area 11: Fuel Management represent?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)

Below is an input that describes a question or task. Write a response that appropriately completes the request.

### Input:
What does Qualified-minus (Q-) performance in Area 11: Fuel Management represent?

### Response:
Qualified-minus (Q-) performance in Area 11: Fuel Management signifies that fuel management was adequate, but there were some minor deviations from standard procedures<|im_end|>


##Loading Finetuned Model from huggingface

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "APP04/qwen-1.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Qwen2 patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/73.9M [00:00<?, ?B/s]

Unsloth 2024.8 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)

Below is an input that describes a question or task. Write a response that appropriately completes the request.

### Input:
What does Qualified-minus (Q-) performance in Area 11: Fuel Management represent?

### Response:
Qualified-minus (Q-) performance indicates that fuel management was not executed correctly, resulting in minor deviations from established procedures<|im_end|>
